In [7]:
source("../Data Generator.r")
library(fuzzyforest)
library(WGCNA)

In [8]:
var_re = 3
### training, validation and test set ###
set.seed(100)
n = 200
p = 400
imp_mod = c(1,4)
var_noise = 1
data = sim_time(n=n,p=p,imp_mod=imp_mod,var_noise=var_noise,a1=0,a2=0,var_re=var_re)

# test set (used for testing performance using optimal parameters)
set.seed(101)
n_test = 100
data_test = sim_time(n=n_test,p=p,imp_mod=imp_mod,var_noise=var_noise,a1=0,a2=0,var_re=var_re)

In [9]:
n_run = 50 # the number of times FF will run on the data set
n_top = 10 # the top n_top variables will be selected
# create empty data frame to save simulation results in
result_ff = matrix(0,n_run+1,p+1) # the last row is for average
result_ff = data.frame(result_ff)
names(result_ff)[p+1] = "error"
names(result_ff)[1:p] = paste("V",1:p,sep="")

In [ ]:
system.time({
for(Repeat in 21:n_run){
    set.seed(Repeat+32) # change seed each loop
    
    var = c(paste("V",1:p,sep=""))
    Formula = as.formula(paste("y~",paste(var,collapse = "+")))
    
    ff <- wff(Formula,data=data,
        screen_params=screen_control(min_ntree = 500,keep_fraction = 0.05),
        select_params = select_control(min_ntree = 500,number_selected = n_top),
        final_ntree = 1000, num_processors = 1)

    top_variables = ff$feature_list[,1]

    # error on the test set
    preds <- predict(ff, new_data=data_test)
    error = mean((data_test$y-preds)^2)

    # If variable was selected as important, indicate with 1 (otherwise 0)
    for (i in 1:p){
      result_ff[Repeat,i] <- as.numeric(paste("V",i,sep="") %in% top_variables)
    }
    result_ff[Repeat,p+1] <- error

    # show the progress
    flush.console()
    cat(Repeat,"\n")
}
})

21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 


In [ ]:
result_ff[n_run+1,] = colMeans(result_ff[1:n_run,])
name = paste("ff_n",n,".csv",sep="")
write.csv(result_ff,file = name)

In [ ]:
result_ff

In [21]:
# sort(result_ff[n_run+1,][1:(p)],index.return=TRUE,decreasing = TRUE)[1:20]